In [1]:
import plotly.graph_objects as go 
import pandas as pd
import numpy as np
from ipywidgets import widgets
# from PIL import Image

# img = Image.open('swz-logotreatment2.png')

df = pd.read_csv('Dice_Rolls.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
str_cols = ['Phr', 'pe', 'ph']
for col in str_cols:
    df[col] = df[col].str.strip("[]")

i = 0
for i in range(len(df)):
#     if i >= 24:
#         print(round(i/599, 3)*100, "% done")
    df['Phr'][i] = " ".join(df['Phr'][i].split()) #this is so ridiculous
    df['Phr'][i] = [float(s) for s in df['Phr'][i].split(" ")]
    df['pe'][i] = " ".join(df['pe'][i].split())
    df['pe'][i] = [float(s) for s in df['pe'][i].split(" ")]
    df['ph'][i] = " ".join(df['ph'][i].split())
    df['ph'][i] = [float(s) for s in df['ph'][i].split(" ")]
    
df.columns

def expected_value(x, y):
    exv = 0
    for i in range(len(x)):
        holder = x[i]*y[i]
        exv = exv + holder
    return(exv)

def inv_cdf(y):
    cdf = [1]
    for i in range(len(y)):
        if i > 0:
            cdf.append(cdf[i-1]-y[i-1])
    return(cdf)


atk_dice = widgets.IntSlider(
    value = 1.0,
    min = 1.0,
    max = 5.0,
    step = 1.0,
    description = 'Attack Dice',
    continuous_update = False)

def_dice = widgets.IntSlider(
    value = 1.0,
    min = 1.0,
    max = 5.0,
    step = 1.0,
    description = "Defense Dice",
    continuous_update = False)

num_ev = widgets.IntSlider(
    value = 0.0,
    min = 0.0,
    max = 2.0,
    step = 1.0,
    description = "Evade Tokens",
    continuous_update = False)

tgt_lk = widgets.Checkbox(
    description = 'Target Lock',
    value = False)

atk_f = widgets.Checkbox(
    description = 'Attack Focus',
    value = False)

def_f = widgets.Checkbox(
    description = 'Defense Focus',
    value = False)

container1 = widgets.HBox(children = [atk_dice, def_dice])
container2 = widgets.HBox(children = [atk_f, tgt_lk])
container3 = widgets.HBox(children = [def_f, num_ev])

j = 1
row = df.iloc[j]
M = (row['M_atk_dice'])
mPhr = row.Phr
# print(mPhr)
mhits = np.arange(M+1)
trace1 = go.Bar(
    x = mhits, 
    y = mPhr, 
    opacity= 0.8, 
    name = 'probability', 
    marker_color = 'indianred')

exv = expected_value(mhits, mPhr)

trace2 = go.Scatter(
    x  = [exv,exv],
    y  = [0,1], 
    name = 'expected # hits',
    line=dict(
        color='firebrick', 
        width=4,
        dash='dash'
    )
)

trace3 = go.Scatter(
    x  = mhits,
    y  = [1]*M, 
    name = 'prob of at least X hits',
    line=dict(
        color='coral', 
        width=4,
        dash='dot'
    )
)


g = go.FigureWidget(data = [trace1, trace2, trace3], 
                    layout = go.Layout(
                        title = dict(text = 'X-Wing Probability of Resolved Hits'),
#                         images =  dict(
#                             source="https://github.com/m-0day/XWing_Dice_Sim/blob/master/swz-logotreatment2.png"
#                         ) 
                    )
                   )
g.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 1.0,
        title_text = "Number of Hits"
    ),
    yaxis = dict(
    title_text = "Probability")
)

# g.add_layout_image(
#     dict(
#         source=img,
#         xref="paper", yref="paper",
#         x=1, y=1.05,
#         sizex=0.2, sizey=0.2,
#         xanchor="right", yanchor="bottom"
#     )
# )
# widgets.VBox([container1, container2, g])

# f = go.FigureWidget()

# 'Phr', 'Ph', 'Pe', 'atk_tl', 'atk_f', 'def_f', 'def_num_ev', 'EV_res', 'atk_ev', 'def_ev', 'pe', 'ph'

def response(change):
#     print(change)

    filter_list = [i and j and k and l and m and n for i, j, k, l, m, n in
                    zip(df['M_atk_dice'] == atk_dice.value, 
                        df['N_def_dice'] == def_dice.value,
                        df['atk_tl'] == tgt_lk.value, 
                        df['atk_f'] == atk_f.value,
                        df['def_f'] == def_f.value,
                        df['def_num_ev'] == num_ev.value)]
    temp_df = df[filter_list]
    
#     print(temp_df)
    M = temp_df['M_atk_dice']
    x1 = np.arange(int(M)+1)
    y1 = temp_df.Phr
    y1 = np.array(list(y1), dtype=np.float)[0]
    y1 = y1[::-1]
    
    exv = expected_value(x1, y1)
    cdf = inv_cdf(y1)
#     print(cdf)
      
#     print("M = ", M, "x1 = ", x1, "y1 = ", y1)
    
    with g.batch_update():
        g.data[0].x = x1
        g.data[0].y = y1
        g.data[1].x = [exv, exv]
        g.data[2].x = x1
        g.data[2].y = cdf

atk_dice.observe(response, names = "value", type = "change")
def_dice.observe(response, names = "value", type = "change")
tgt_lk.observe(response, names="value", type = "change")
atk_f.observe(response, names="value", type = "change")
def_f.observe(response, names="value", type = "change")
num_ev.observe(response, names="value", type = "change")

widgets.VBox([container1, container2, container3, g])

C:\Users\CM\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\CM\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\CM\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\CM\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFr